In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
spark = SparkSession.builder.appName("gdelt-trends").getOrCreate()

23/12/20 22:56:04 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
df = spark.read.csv(path="gs://dataproc-staging-us-east1-1077714986221-w3r4k84a/table3",header=True)

23/12/20 22:56:24 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/12/20 22:56:39 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/12/20 22:56:54 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/12/20 22:57:09 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/12/20 22:57:24 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/12/20 22:57:39 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registere

In [25]:
df.show(2)

+----------+----------------+---------+----------+--------------------+--------------------+
|      Date|SourceCommonName|Sentiment|      news|            v2counts|         v2locations|
+----------+----------------+---------+----------+--------------------+--------------------+
|2023-08-30|     jutarnji.hr|    -6.87|   poverty|AFFECT#2000000#me...|3#Keaton Beach, F...|
|2023-08-30|   eastmoney.com|     5.13|epu_policy|SEIZE#2#frame#3#P...|1#China#CH#CH##35...|
+----------+----------------+---------+----------+--------------------+--------------------+
only showing top 2 rows



In [3]:
from pyspark.sql.functions import col, to_date, mean, count, weekofyear, date_trunc
from pyspark.sql import Window

def creating_dataset(df, column):
    '''
    Column is the column in which are allocated the keywords, for every case: political, social and economical columns
    '''
    # Convert 'Date' column to date type
    df = df.withColumn('Date', to_date(col('Date')))

    # List of new columns (keywords)
    list_keywords = [row[column] for row in df.select(column).distinct().collect()]
    df_final = df.select('Date').distinct().withColumnRenamed('Date', 'date')

    for k in list_keywords:
        # Creating a new dataframe for every keyword in the column, getting the occurrences of keyword and mean of sentiment
        df_keyword = df.filter(df[column] == k).groupBy('Date').agg(count('*').alias('count'), mean('Sentiment').alias('mean'))
        
        # This will be our score, occurrences * mean
        df_keyword = df_keyword.withColumn(k, col('count') * col('mean'))

        # Date column to perform the join by it
        df_keyword = df_keyword.withColumnRenamed('Date', 'date')

        # Join with the main dataframe
        df_final = df_final.join(df_keyword.select('date', k), on='date', how='left')

    # grouping by week
    window = Window.orderBy(col('date'))
    df_final = df_final.withColumn('date', date_trunc('week', col('date'))).groupBy('date').mean()

    for column in df_final.columns:
        if column.startswith('avg('):
            new_column_name = column[4:-1]  # Remove 'avg(' at start and ')' at the end
            df_final = df_final.withColumnRenamed(column, new_column_name)

    # Sort by date and reset index
    df_final = df_final.orderBy('date')

    return df_final

In [4]:
df_gdelt_news = df.orderBy("Date")
df_new = creating_dataset(df_gdelt_news,"news")

In [ ]:
#Created our feature dataframe by aggregating Theme Sentiment Scores per week by calling the creating_dataset function

In [5]:
df_new = df_new.withColumnRenamed("date","timestamp_date")
df_new = df_new.withColumn("date", col("timestamp_date").cast("date"))
df_new = df_new.drop("timestamp_date")

In [7]:
df_new = df_new.orderBy("Date")
df_new.show(5)

23/12/20 23:04:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------------------------+-------------------+-------------------+-------------+-------------------+------------------------+------------------+-------------------+-------------------+--------------------------+-------------------+------------------+-------------------+----------+
|job_quality_&_labor_market_performance|            poverty|         bankruptcy|central_banks|       stock_market|health_economics_finance|        epu_policy|          oil_price|    economic_growth|financial_arch_and_banking| Debt_Vulnerability|         inflation|    econ_free_trade|      date|
+--------------------------------------+-------------------+-------------------+-------------+-------------------+------------------------+------------------+-------------------+-------------------+--------------------------+-------------------+------------------+-------------------+----------+
|                    -124.2433333333333| -242.0266666666666|-13.059999999999999|         null|            -399.6

In [10]:
df_google = spark.read.csv(path="gs://dataproc-staging-us-east1-1077714986221-w3r4k84a/unemployment_pytrends-2020.csv",header=True)

In [11]:
df_google = df_google.orderBy("date")
df_google_dates = df_google.select("date").distinct()
df_google_dates = df_google_dates.withColumn("date", F.to_date(F.col("date"))).orderBy("date")

In [12]:
keyword_list = [row['keyword'] for row in df_google.select('keyword').distinct().collect()]
keyword_list.sort()

In [ ]:
#Renaming column from trend_index to Unemployment

In [13]:
for k in keyword_list:
    df_google_dates = df_google_dates.join(df_google.filter(df_google['keyword'] == k).select('date', 'trend_index').withColumnRenamed('trend_index', k), 'date', 'left')

In [14]:
df_google_dates = df_google_dates.withColumn("unemployment", col("unemployment").cast("double"))
df_google_dates.printSchema()

root
 |-- date: date (nullable = true)
 |-- unemployment: double (nullable = true)



In [26]:
df_google_dates.show(5)

+----------+------------+
|      date|unemployment|
+----------+------------+
|2020-01-05|         5.0|
|2020-01-12|         5.0|
|2020-01-19|         5.0|
|2020-01-26|         5.0|
|2020-02-02|         5.0|
+----------+------------+
only showing top 5 rows



In [ ]:
#Manually defining the start and end dates, ideally this should be extracted from the data being considered

In [15]:
from datetime import datetime, timedelta
from pyspark.sql import Row
date1 = datetime.strptime('2020-12-28', '%Y-%m-%d')
max_date = datetime.strptime('2023-11-30', '%Y-%m-%d')
date_list = [date1 + timedelta(days=x) for x in range(0, (max_date - date1).days + 1, 7)]
df_f = spark.createDataFrame([Row(date=d) for d in date_list])
df_f = df_f.withColumnRenamed("date","timestamp_date")
df_f = df_f.withColumn("date", col("timestamp_date").cast("date"))
df_f = df_f.drop("timestamp_date")

In [ ]:
#Joining the features and the prediction column on the date column

In [16]:
df_final = df_f
datasets = [df_new,df_google_dates]
for d in datasets:
    df_final = df_final.join(d,'date','left_outer')

In [17]:
df_final.printSchema()

root
 |-- date: date (nullable = true)
 |-- job_quality_&_labor_market_performance: double (nullable = true)
 |-- poverty: double (nullable = true)
 |-- bankruptcy: double (nullable = true)
 |-- central_banks: double (nullable = true)
 |-- stock_market: double (nullable = true)
 |-- health_economics_finance: double (nullable = true)
 |-- epu_policy: double (nullable = true)
 |-- oil_price: double (nullable = true)
 |-- economic_growth: double (nullable = true)
 |-- financial_arch_and_banking: double (nullable = true)
 |-- Debt_Vulnerability: double (nullable = true)
 |-- inflation: double (nullable = true)
 |-- econ_free_trade: double (nullable = true)
 |-- unemployment: double (nullable = true)



In [18]:
df_final = df_final.fillna(0)

In [19]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.stat import ChiSquareTest
from pyspark.sql.functions import col

In [20]:
features_cols = [col for col in df_final.columns if col not in ["date", "unemployment"]]
assembler = VectorAssembler(inputCols=features_cols, outputCol="features")
df_vector = assembler.transform(df_final)

In [21]:
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scalerModel = scaler.fit(df_vector)
df_scaled = scalerModel.transform(df_vector)

In [23]:
train_df, test_df = df_scaled.randomSplit([0.8, 0.2], seed=1234)

In [22]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol="scaled_features", labelCol="unemployment", regParam=0.1, elasticNetParam=0.5)

In [ ]:
lrModel = lr.fit(train_df)
predictions = lrModel.transform(test_df)

In [ ]:
predictions.write.parquet("gs://dataproc-staging-us-east1-1077714986221-w3r4k84a/outputs/predictions_b69e.parquet")

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="unemployment", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

In [ ]:
evaluator.setMetricName("mse")
mse = evaluator.evaluate(predictions)
print("Mean Squared Error (MSE) on test data = %g" % mse)

In [ ]:
evaluator.setMetricName("mae")
mae = evaluator.evaluate(predictions)
print("Mean Absolute Error (MAE) on test data = %g" % mae)